In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%watermark -v -p numpy,pandas,matplotlib,sklearn,torch,torchvision,pytorch_lightning

CPython 3.6.9
IPython 7.16.1

numpy 1.18.5
pandas 1.0.4
matplotlib 3.2.1
sklearn 0.23.1
torch 1.6.0.dev20200609+cu101
torchvision 0.7.0.dev20200609+cu101
pytorch_lightning 0.8.5


In [19]:
import warnings

import json
import os
from collections import OrderedDict
import numpy as np
import pandas as pd
import torch
import torchvision
from PIL import Image

import torch.nn.functional as F
from torch.optim import SGD, Adam
from torch.nn import NLLLoss, MSELoss, CrossEntropyLoss
from torch.nn import Module, Conv2d, Dropout2d, Linear
from torch.optim.lr_scheduler import ExponentialLR, StepLR, MultiStepLR, ReduceLROnPlateau
from torch.utils.data import (Dataset, DataLoader)
from torchvision import transforms


warnings.filterwarnings('ignore')

from k12libs.utils.nb_easy import (EasyaiClassifier, EasyaiTrainer)

---------------------------------------

## 简单实例

该实例没有实际意义, 默认配置的是 resnet18网络, 使用rmnist数据集

In [3]:
trainer = EasyaiTrainer(max_epochs=2)
trainer.fit(EasyaiClassifier())
trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11 M  


--------------------------------------------------------------------------------
TEST RESULTS
{'test_acc': tensor(0.9236, device='cuda:0'),
 'test_loss': tensor(0.2514, device='cuda:0')}
--------------------------------------------------------------------------------



{'test_loss': 0.2513880133628845, 'test_acc': 0.9236111640930176}

## 自定义: 修改默认预置模型和预置数据集

继承EasyaiClassifier, 实现prepare_data和build_model.

In [18]:
# class CustomClassifier(EasyaiClassifier):
#     # 修改: 使用预置的'rchestxray'数据集
#     def prepare_dataset(self):
#         # 返回dict: {'train': Dataset, 'val': Dataset, 'test': Dataset}
#         return self.load_presetting_dataset_('chestxray')
#     
#     # 修改: 使用预置的'shufflenetv2'模型
#     def build_model(self):
#         return self.load_pretrained_model_('shufflenet_v2_x0_5', num_classes=2)

In [ ]:
# trainer = EasyaiTrainer(max_epochs=10)
# # 训练
# trainer.fit(CustomClassifier())
# # 评估
# trainer.test()

## 自定义: 构建用户模型, 加载用户数据集

继承EasyaiClassifier, 实现build_model(不适用预置模型)和prepare_data(自定义数据集解析处理)

In [17]:
# class CustomClassifier(EasyaiClassifier):
#     
#     # 预处理数据集
#     def prepare_dataset(self):
#         # 使用json文件描述数据集的情况
#         class JsonfileDataset(Dataset):
#             def __init__(self, root, phase, info):
#                 self.root = root
#                 self.info = info
#                 image_list = []
#                 label_list = []
#                 with open(os.path.join(self.root, f'{phase}.json')) as f:
#                     items = json.load(f)
#                     for item in items:
#                         image_list.append(os.path.join(self.root, item['image_path']))
#                         label_list.append(item['label'])
#                 self.image_list, self.label_list = image_list, label_list
#                 
#                 self.augtrans = None
#                 self.imgtrans = transforms.Compose([
#                     transforms.ToTensor(),
#                     transforms.Normalize(mean=info['mean'], std=info['std'])
#                 ])
#                 
#             # 实现data_augment方法
#             def data_augment(self, augtrans):
#                 self.augtrans = transforms.Compose(augtrans)
#                 
#             def __getitem__(self, index):
#                 img = Image.open(self.image_list[index]).convert('RGB')
#                 if self.augtrans:
#                     img = self.augtrans(img)
#                 img = self.imgtrans(img)
#                 return img, self.label_list[index]
#             def __len__(self):
#                 return len(self.image_list)
# 
#         # 数据集存放目录
#         dataroot = os.path.join('/data/datasets/cv/', 'rmnist')
#         with open(os.path.join(dataroot, 'info.json')) as f:
#             info = json.load(f)
#             
#         return {
#             'train': JsonfileDataset(dataroot, 'train', info),
#             'val': JsonfileDataset(dataroot, 'val', info),
#             'test': JsonfileDataset(dataroot, 'test', info),
#         }
#     
#     # 构建模型
#     def build_model(self):
#         class CustomNet(Module):
#             def __init__(self):
#                 super(CustomNet, self).__init__()
#                 self.conv1 = Conv2d(3, 32, 3, 1)  # 卷积层, 图片特征提取
#                 self.conv2 = Conv2d(32, 64, 3, 1)
#                 self.dropout1 = Dropout2d(0.25)   # Dropout正则化, 减少模型过拟合
#                 self.fc1 = Linear(9216, 128)
#                 self.dropout2 = Dropout2d(0.5)
#                 self.fc2 = Linear(128, 10)        # 全连接层, 图片线性变换
#             def forward(self, x):
#                 x = self.conv1(x)
#                 x = F.relu(x)
#                 x = self.conv2(x)
#                 x = F.relu(x)
#                 x = F.max_pool2d(x, 2)
#                 x = self.dropout1(x)
#                 x = torch.flatten(x, 1)
#                 x = self.fc1(x)
#                 x = F.relu(x)
#                 x = self.dropout2(x)
#                 x = self.fc2(x)
#                 # x = F.log_softmax(x, dim=1)
#                 return x
#         return CustomNet()

In [ ]:
# trainer = EasyaiTrainer(max_epochs=10)
# # 训练
# trainer.fit(CustomClassifier())
# # 评估
# trainer.test()

## 自定义: 用户自定义loss, optimize, schedule

继承EasyaiClassifier, 实现configure_criterion, configure_optimizer, configure_scheduler

In [ ]:
# class CustomClassifier(EasyaiClassifier):
#    # 配置损失函数: CrossEntropyLoss(交叉熵损失)
#    def configure_criterion(self):
#        loss = CrossEntropyLoss(
#            reduction='mean' # 约简方式: 张量各个维度上的元素的平均值
#        )
#        return loss
#    
#    # 配置优化方法: 随机梯度下降(SGD)
#    def configure_optimizer(self):
#        # self.model是在build_model构造的, 如果build_model没定义, 使用默认的构造的预置模型
#        optimizer = SGD(
#            filter(lambda p: p.requires_grad, self.model.parameters()),
#            lr=0.01,           # 基础学习率
#            weight_decay=1e-6, # 权重衰减, 使得模型参数值更小, 有效防止过拟合
#            momentum=0.9,      # 动量因子, 更快局部收敛
#            nesterov=True      # 使用Nesterov动量, 加快收敛速度
#        )
#        return optimizer
#    
#    # 配置学习率策略: 固定步长衰减(StepLR)
#    def configure_scheduler(self, optimizer):
#        # optmizer是在configure_optimizer配置的
#        scheduler = StepLR(
#            optimizer,   # 优化器
#            step_size=2, # 每间隔2次epoch进行一次LR调整
#            gamma=0.6    # LR调整为原来0.6倍
#        )
#        return scheduler

In [16]:
# trainer = EasyaiTrainer(max_epochs=10)
# # 训练
# trainer.fit(CustomClassifier())
# # 评估
# trainer.test()

## 自定义: 数据增强

继承EasyaiClassifier, 实现train_dataloader, val_dataloader, test_dataloader

In [ ]:
# class CustomClassifier(EasyaiClassifier):
#     # 训练时进行数据增强
#     def train_dataloader(self) -> DataLoader:
#         # self.datasets是在prepare_data方法中生成, 如果没有自定义prepare_data, 使用默认预置数据集
#         dataset = self.datasets['train']
#         dataset.data_augment([
#             transforms.Resize((64, 64)),      # 改变图片的大小
#             transforms.RandomHorizontalFlip() # 让图片进行水平翻转                    
#         ])
#         return DataLoader(
#             dataset,
#             batch_size=64,    # 每次输入模型的图片个数(批量大小)
#             drop_last=False,  # 最后一次的图片数量不等于设置的batch_size是否丢弃
#             num_workers=2,    # 启动多个进程加载数据集, 不可设置过大
#             shuffle=True,     # 送入模型的图片是否进行随机打散
#         )
#     
#     # 校验时进行数据增强 (同train_dataloader)
#     def val_dataloader(self) -> DataLoader:
#         dataset = self.datasets['val']
#         dataset.data_augment([
#             transforms.Resize((64, 64)),
#         ])
#         return DataLoader(dataset, batch_size=64)
#     
#     # 评估时不进行数据增强
#     def test_dataloader(self) -> DataLoader:
#         dataset = self.datasets['test']
#         return DataLoader(dataset, batch_size=64)

In [ ]:
trainer = EasyaiTrainer(max_epochs=10)
# 训练
trainer.fit(CustomClassifier())
# 评估
trainer.test()

----------------------------------------

## 接口

In [14]:
class CustomClassifier(EasyaiClassifier):

    ##########################################################################
    ####### Data ######
    ##########################################################################
    def load_presetting_dataset_(self, dataset_name):
        """
        加载平台预置的数据集
        Args:
            dataset_name: 数据集的名字, 如: rmnist, flowers等
        
        Return:
            以下几种方式任意一种:
            1. EasyaiDataset实例, 表明只进行训练(只返回了训练数据集实例)
            2. EasyaiDataset实例列表, 当列表长度为2时, 说明还要进行训练的校验, 当列表长度为3时, 说明还要进行测试评估.
            3. EasyaiDataset实例字典, 如: {'train': EasyaiDataset, 'val': EasyaiDataset, 'test':EasyaiDataset}
        """
        pass
    
    def prepare_dataset(self):
        """
        准备数据集, 从磁盘上加载数据集, 不同数据集的描述格式可能不一样, 一般有json/xml/csv等描述格式,
        也可能直接是图片目录, 所有这些格式的处理可以在这个接口完成.
            
        Return:
            同prepare_dataset
        """
        pass
    
    ##########################################################################
    ####### Model ######
    ##########################################################################
    def load_pretrained_model_(self, model_name, num_classes, pretrained):
        """
        加载平台预置的模型
        Args:
            model_name: 模型的名字
            num_classes: 分类个数(数据集labels的数目), 预置的模型默认是1000, 所以需要提供真实的分类个数进行处理
            pretrained: 是否加载预置权重
            
        Return:
            Module: 模型实例
        """
        pass
    
    def build_model(self):
        """
        构建模型, 可以自定义模型, 也可以调用load_pretrained_model_使用平台预置的模型
        
        Return:
            同load_pretrained_model_
        """
        pass
    
    
    ##########################################################################
    ####### Hypes Parameters ######
    ##########################################################################
    def configure_criterion(self):
        """
        配置损失函数
        
        Return:
            loss
        """
        pass
    
    def configure_optimizer(self):
        """
        配置优化器
        
        Return:
            optimizer
        """
        pass

    def configure_scheduler(self, optimizer):
        """
        配置学习率衰减策略
        
        Args:
            optimizer: 优化器(通过configure_optimizer配置得到的)
        
        Return:
            scheduler
        """
        pass
    
    def loss(self):
        """
        返回configure_criterion配置的损失实例
        """
        pass

    
    ##########################################################################
    ####### Trainer: Train ######
    ##########################################################################
    def train_dataloader(self):
        """
        训练数据集批量控制加载器, 可以设置批量的大小, 是否对数据进行洗牌(shuffle)等
        
        Return:
            loader
        """
        pass
    
    def training_step(self, batch, batch_idx):
        """
        训练过程中, 迭代一次batch数据, 就会触发一次training_step的调用,训练,统计metrics
        
        Args:
            batch: 一个batch的数据内容, 一般包括图片(image), 图片标签(labels), 图片路径(path).
                具体batch中内容受prepare_dataset接口的实现会有所不同
            batch_idx: 批量迭代次数, 该数值一直累加, 不受下一次epoch的影响
            
        Return:
            metrics: 必须包含loss关键字, log(日志模块)和progress_bar(进度条显示)是可选的
        """
        pass
    
    def training_epoch_end(self, outputs):
        """
        训练过程中, 每完整的遍历完一次(epoch)数据集,则触发一次training_epoch_end的调用, 汇总metrics
        
        Args:
            outputs: 所有metrics(training_step每次产生的数据的集合)
        
        Return:
            metrics: 同training_step, 可以对metrics做平均处理
        """
        pass

    ##########################################################################
    ####### Trainer: validation ######
    ##########################################################################
    def val_dataloader(self):
        """
        同train_dataloader
        """
        pass
    
    def validation_step(self, batch, batch_idx):
        """
        同train_step
        """
        pass
        
    def validation_epoch_end(self, outputs):
        """
        同train_epoch_end
        """
        pass
    
    ##########################################################################
    ####### Trainer: test ######
    ##########################################################################
    def test_dataloader(self):
        """
        同train_dataloader
        """
        pass

    def test_step(self, batch, batch_idx):
        """
        同train_step
        """
        pass
        
    def test_epoch_end(self, outputs):
        """
        同train_epoch_end
        """
        pass

## 实例: 预置数据集和预置模型

In [15]:
class CustomClassifier(EasyaiClassifier):
    def prepare_dataset(self):
        """
        加载预设数据集:chestxray (胸部xray判断新冠)
        """
        return self.load_presetting_dataset_(dataset_name='chestxray')

    def build_model(self):
        """
        加载预置模型: densenet201
        """
        return self.load_pretrained_model_(model_name='densenet201', num_classes=2, pretrained=True)

trainer = EasyaiTrainer(max_epochs=10)
trainer.fit(CustomClassifier())

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | DenseNet | 18 M  


1

## 实例: 修改损失函数, 优化器, LR策略

In [24]:
class CustomClassifier(EasyaiClassifier):
    # 配置损失函数: CrossEntropyLoss(交叉熵损失)
    def configure_criterion(self):
        loss = CrossEntropyLoss(
            reduction='mean' # 约简方式: 张量各个维度上的元素的平均值
        )
        return loss
   
    # 配置优化方法: 随机梯度下降(SGD)
    def configure_optimizer(self):
        # self.model是在build_model构造的, 如果build_model没定义, 使用默认的构造的预置模型
        optimizer = SGD(
            filter(lambda p: p.requires_grad, self.model.parameters()),
            lr=0.01,           # 基础学习率
            weight_decay=1e-6, # 权重衰减, 使得模型参数值更小, 有效防止过拟合
            momentum=0.9,      # 动量因子, 更快局部收敛
            nesterov=True      # 使用Nesterov动量, 加快收敛速度
        )
        return optimizer
    
    # 配置学习率策略: 
    def configure_scheduler(self, optimizer):
        # optmizer是在configure_optimizer配置的
        
        # policy-1: 指数衰减
        # scheduler = ExponentialLR(optimizer, gamma=0.5)
        
        # policy-2: 指标不在变化时, 调整学习率
        # scheduler = ReduceLROnPlateau(
        #     optimizer,   # 优化器
        #     mode='min',  # 指定指标不再下降
        #     factor=0.1,  # 衰减因子
        #     patience=6,  # 容忍多少次(指标不改变)
        #     eps=1e-6,    # 学习率衰减到的最小值eps时,学习率不再改变
        # )
        
        # policy-3: 固定步长衰减
        scheduler = StepLR(
            optimizer,   # 优化器
            step_size=2, # 每间隔2次epoch进行一次LR调整
            gamma=0.6    # LR调整为原来0.6倍
        )
        
        # policy-4: 不定步长衰减
        # scheduler = MultiStepLR(
        #     optimizer,        # 优化器
        #     milestones=[3, 7],# 分阶段(epoch)调整,到达指定的epoch时, 学习率调整为原来的gamma倍
        #     gamma=0.1         # LR调整为原来0.1倍
        # )
        return scheduler
    
trainer = EasyaiTrainer(max_epochs=10)
trainer.fit(CustomClassifier())

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11 M  


1

## 实例: 修改训练过程及训练Metrics

In [26]:
from k12libs.utils.nb_easy import (EasyaiClassifier, EasyaiTrainer)

class CustomClassifier(EasyaiClassifier):
    # 进入模型前, 数据加载参数设置
    def train_dataloader(self) -> DataLoader:
        return self.get_dataloader('train',
                batch_size=48,    # 一次进入模型的数据量(批量大小)
                num_workers=1,    # 启动多少个进程加载数据(值不要过大)
                drop_last=True,   # 是否丢大最后一次不完整的batch(个数不足batch_szie)
                shuffle=False)    # 每次epoch时,是否将数据进行重新洗牌
    
    # 对每次batch迭代, 计算损失率, 正确率等, 返回metrics 
    def training_step(self, batch, batch_idx):
        x, y, p = batch           # images, labels, paths      
        y_hat = self.model(x)     # model由build_model返回, 输入images, 返回估计y
        loss = self.loss(y_hat, y)# loss有configure_criterion配置, 计算y_hat, y的差异损失
        with torch.no_grad():
            acc = (torch.argmax(y_hat, axis=1) == y).float().mean() # 计算正确率
            
        log = {'train_loss': loss, 'train_acc': acc}
        output = OrderedDict({
            'loss': loss,        # (M) 必须包含loss, 训练损失
            'acc': acc,          # (O) 可选, 每次迭代记录正确率, epoch结束时可以用来计算平均acc
            'progress_bar': log, # (O) 可选, 可以在训练进度条上显示该字典里的内容
            "log": log           # (O) 可选, 如果启动logger模块, 作为logger的输入数据
        })
        return output

    # 对每次eopch的结束, 根据需要计算metrics, 输出日志等
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean() # 对训练loss做平均
        avg_acc = torch.stack([x['acc'] for x in outputs]).mean()   # 对训练acc做平均
        log = {'train_loss': avg_loss, 'train_acc': avg_acc}        # 封装到字典中
        return {'progress_bar': log, 'log': log}                    # 供进度条上和logger模块使用log字段数据
     
trainer = EasyaiTrainer(max_epochs=10)
trainer.fit(CustomClassifier())

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11 M  


1

In [ ]:
## 实例: 